## Mathematical Foundations of Computer Graphics and Vision 2023
## Exercise 2 - Global Optimization

In this exercise you will apply what you learned about global optimization, especially branch and bound (B&B), concave and convex envelopes, and reformulation. You will implement branch and bound for consensus set maximization.

<b style="color:red"> TODO A: </b>:  Derivation of the problem formulation in the canonical form of Linear Programming. Please explain all your steps clearly. You may use the hints and notation from section 2 of the excercise sheet.

Your answer here (Type *Markdown* and LaTeX: $\alpha^2$)

#### Branch and Bound Implementation for the Consensus Set Maximization Problem

<b style="color:red"> TODO B: </b>In the following code you are supposed to implement consensus set maximization by branch and bound in the context of stereo matching where the model is a 2D translation between two input images. Please use the imports listed in the cell down. You may also add more imports for example for visualization purposes if you find them useful.


In [ ]:
%matplotlib inline
import math
import heapq
from typing import Tuple, List, Optional
from collections import namedtuple
import os

import numpy as np
from scipy.io import loadmat
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator


# Translation Theta
T = namedtuple("T", ["x", "y"])

We provide you some skeleton code that you are free to use as it is or where you can also add/ delete parameters.
You may also write auxiliary functions if you feel the need to do so.

In [ ]:
# Load input points, input image and fix delta parameter
corr = loadmat('data/ListInputPoints')['ListInputPoints'] # correspondences (nx4 array) [x_i,y_i,x_i',y_i']
N = corr.shape[0]  # number of total points

left_image = plt.imread('data/InputLeftImage.png')
right_image = plt.imread('data/InputRightImage.png')
image_size = left_image.shape

delta = 3  # inlier threshold [pixels]

In [ ]:
def plot_matches(inliers: List, outliers: List):
    fig, ax = plt.subplots(1, figsize=(17, 7))
    
    gap = np.zeros_like(left_image[:, :20])
    stitched = np.concatenate((left_image, gap, right_image), 1)
    ax.imshow(stitched)
    
    offset = left_image.shape[1] + gap.shape[1]

    for idx, line in enumerate(corr):
        x1, y1, x2, y2 = line
        if idx in inliers:
            color = "lime"
        elif idx in outliers:
            color = "crimson"
        else:
            color = "blue"
        
        ax.plot([x1, offset + x2], [y1, y2], ".-", color=color, linewidth=0.8)

    ax.axis('off')
    plt.show(fig)
    plt.close(fig)


plot_matches(inliers=[], outliers=[])

To help with implementing and debugging BnB, we provide a naive estimation of the inlier upper bound within box constraints of $[\underline{T_x}, \overline{T_x}]\times[\underline{T_y}, \overline{T_y}]$.

In [ ]:
def naive_upper_bound(theta_lower: T, theta_upper: T) -> Tuple[T, int]:
    """
    Naive estimation of the upper bound on objective
    Calculates number of inliers in region [Tx_lb, Tx_ub]x[Ty_lb, Ty_ub]

    Parameters:
    - theta_lower, theta_upper: Lower and upper bounds for T

    Returns:
    - Tuple (translation, objcost) representing the estimated upper bound on the model and the objective
    """

    x1, y1, x2, y2 = corr.T
    
    T_x = x2 - x1
    T_y = y2 - y1
    
    inliers = (((T_x + delta) >= theta_lower.x) *
               ((T_y + delta) >= theta_lower.y) *
               ((T_x - delta) <= theta_upper.x) *
               ((T_y - delta) <= theta_upper.y))
    
    #Set center of region as new model
    theta = T(0.5 * (theta_lower.x + theta_upper.x), 0.5 * (theta_lower.y + theta_upper.y))

    return theta, np.sum(inliers)


In [ ]:
def solve_relaxed_LP(theta_lower: T, theta_upper: T) -> Tuple[T, int]:
    """
    Parameters:
    - theta_lower, theta_upper: Lower and upper bounds for T
    
    Returns:
    -  Tuple that contains the solution (T, objcost) of the relaxed LP
    """
    
    #TODO: Implement relaxed LP
    theta = T(0, 0)
    num_inliers = 0
    
    return theta, num_inliers


In [ ]:
def is_inlier(theta: T) -> np.ndarray:
    """
    Parameters:
    - theta - model that's being tested
    
    Returns:
    - bool vector f size N, where i-th element indicates if i-th point of `corr` is an inlier
    """

    #TODO: Compute if i-th element is an inliers
    inliers = ...
    
    return inliers


In [ ]:
def consensus_set_maximization_by_bnb(image_size: Tuple[int, int], upper_bound_fn) -> Tuple[T, List[int], List[int]]:
    """
    Parameters:
    - image_size - tuple of image size (H, W)

    Returns a tuple of
    - Best model according to the BnB algorithm
    - list of upper bounds for each iteration
    - list of lower bounds for each iteration
    """
    
    h, w, _ = image_size

    best_model = T(0, 0)
    upper_bounds = []
    lower_bounds = []

    #TODO: Implement BnB

    return best_model, lower_bounds, upper_bounds


In [ ]:
def plot_bound_convergence(lower_bounds: List[int], upper_bounds: List[int]):
    fig, ax = plt.subplots(1, figsize=(17, 7))
    
    iterations = np.arange(len(lower_bounds)) + 1
    
    # Draw lines
    ax.plot(iterations, lower_bounds, marker="o", color="r")
    ax.plot(iterations, upper_bounds, marker="o", color="b")
    
    # Write titles
    ax.set_title("Convergence of Bounds")
    ax.set_xlabel("Iterations")
    ax.set_ylabel("Upper and Lower Bounds")
    ax.legend(["Lower Bound", "Upper Bound"], loc="upper right")
    ax.xaxis.set_major_locator(MultipleLocator(1 if len(lower_bounds) < 20 else 5))
    ax.yaxis.set_major_locator(MultipleLocator(5))
    ax.grid(True)

    plt.show(fig)
    plt.close(fig)


In [ ]:
def perform_bnb(upper_bound_fn):
    
    #perform consensus set maximization
    best_model, lower_bounds, upper_bounds = consensus_set_maximization_by_bnb(image_size, upper_bound_fn)
    
    inliers = is_inlier(best_model)
    
    inliers_indices = np.where(inliers)[0]
    outliers_indices = np.where(~inliers)[0]
    
    print(f"Global optimal translational model (Tx, Ty) = {best_model}")
    print(f"Inlier set S_I: {inliers_indices} ({len(inliers_indices)} points)")
    print(f"Outlier indices S_O = S\S_I: {outliers_indices}")
    
    plot_matches(inliers_indices, outliers_indices)
    
    plot_bound_convergence(lower_bounds, upper_bounds)


#### Results

<b style="color:red"> TODO C-E: </b> Present and discuss your results. Also plot the figures described in the deliverables D and E.

In [ ]:
# Display results with the naive upper bound
perform_bnb(naive_upper_bound)

In [ ]:
# Display results with the LP upper bound
perform_bnb(solve_relaxed_LP)

<b style="color:red"> Discussion (not graded): </b> In the previous exercise, you implemented RANSAC to fit polynomials. Now, can you use RANSAC to solve the consensus set maximization problem for stereo matching? If so, please briefly describe the main steps you would take to implement RANSAC for this task, and compare it with the BnB approach. Additionally, can you identify scenarios where one approach may be more effective than the other? If not, explain why it is not feasible to adapt RANSAC for this application. Can you also combine RANSAC and BnB in one pipeline?

_TODO_: Your answer here